In [ ]:
# project 2d labels onto 3d and calculate IOU
# over train/val set

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import h5py
import numpy as np
import imageio
import torch
import os, os.path as osp
from tqdm import tqdm
import cv2
import torch.nn.functional as F
import random

from eval.common import ConfMat

from datasets.scannet.utils_3d import ProjectionHelper, adjust_intrinsic, make_intrinsic, load_intrinsic, load_pose
from datasets.scannet.utils_3d import load_depth
from datasets.scannet.common import read_label_mapping, map_labels, nyu40_to_continuous
from datasets.scannet.common import VALID_CLASSES

In [ ]:

def get_scan_name(scene_id, scan_id):
    return f'scene{str(scene_id).zfill(4)}_{str(scan_id).zfill(2)}'

# globals
subvol_size = (32, 32, 64)
voxel_size = 0.05
voxel_dims = (1, 1, 1)
root = Path('/mnt/data/scannet/scans')
proj_img_size = (320, 240)
scannet_to_nyu40 = read_label_mapping('/mnt/data/scannet/scannetv2-labels.combined.tsv')
num_classes = 40

data_dir = Path('/mnt/data/scannet/backproj')
# fname = 'train100-v7.h5'
fname = 'val-v8.h5'
f = h5py.File(data_dir / fname, 'r')
f['x'].shape

In [ ]:
f.close()

In [ ]:
# init metrics
confmat = ConfMat(num_classes)
skipped = 0
inds = list(range(len(f['x'])))
random.shuffle(inds)
inds = inds[:2000]
not_occupied = 0

for ndx in tqdm(range(len(f['x']))):
# for ndx in tqdm(range(2000)):
    w2g, sceneid, scanid, frames = f['world_to_grid'][ndx], f['scene_id'][ndx], f['scan_id'][ndx], f['frames'][ndx]

    # ignore subvols that are not occupied (val set)
    subvol_x = f['x'][ndx]
    if (subvol_x == 1).sum() == 0:
        not_occupied += 1
        continue
        
    subvol_y = f['y'][ndx]
    # per-scene basics
    scan_name = get_scan_name(sceneid, scanid)
    frame_ndx = 0
    
    # val set - no frame, then skip
    if frames[frame_ndx] == -1:
        skipped += 1
        continue
        
    pose_path = root / scan_name / 'pose' / f'{frames[frame_ndx]}.txt'
    pose = load_pose(pose_path).numpy()
    depth_path = root / scan_name / 'depth' / f'{frames[frame_ndx]}.png' 
    depth = load_depth(depth_path, proj_img_size)
    # get projection
    intrinsic_path = root / scan_name / 'intrinsic/intrinsic_color.txt'
    intrinsic = load_intrinsic(intrinsic_path)
    # adjust for smaller image size
    intrinsic = adjust_intrinsic(intrinsic, [1296, 968], proj_img_size)

    projection = ProjectionHelper(
                intrinsic, 
                0.4, 4.0,
                proj_img_size,
                subvol_size, voxel_size
            )

    proj = projection.compute_projection(torch.Tensor(depth), torch.Tensor(pose), torch.Tensor(w2g))
    if proj is None: 
        continue
    proj3d, proj2d = proj
    num_inds = proj3d[0]

    ind3d = proj3d[1:1+num_inds]
    ind2d = proj2d[1:1+num_inds]

    # load the label image
    # same as ENet
    label_path = root / scan_name / 'label-filt' / f'{frames[frame_ndx]}.png' 
    label_scannet = np.array(imageio.imread(label_path))
    label_nyu40 = map_labels(label_scannet, scannet_to_nyu40)
    # map from NYU40 labels to 0-39 + 40 (ignored) labels, H,W
    y = nyu40_to_continuous(label_nyu40, ignore_label=num_classes, 
                                        num_classes=num_classes)
    # resize label image here using the proper interpolation - no artifacts  
    # dims: H,W                                     
    y = cv2.resize(y, proj_img_size, interpolation=cv2.INTER_NEAREST)
    y = torch.LongTensor(y.astype(np.int32))
    
    # labels at the required locations, index into H,W image
    labels = y.view(-1)[ind2d]
    # 40/ignore labels in 2d -> dont project 
    invalid_2d = (labels == num_classes)
    valid_2d = torch.logical_not(invalid_2d)
    
    valid_ind2d = ind2d[valid_2d]
    valid_ind3d = ind3d[valid_2d]
    
    labels = y.view(-1)[valid_ind2d]
    
    # get the label volume - DHW
    # create empty volume with zeros
    output = torch.zeros(subvol_size[2], subvol_size[1], subvol_size[0], dtype=int)
    output.view(-1)[valid_ind3d] = labels.T
    # update metric
    subvol_y = torch.LongTensor(subvol_y).permute(2, 1, 0)
    confmat.update(output, subvol_y)
    
# compute metric
class_subset = np.array(VALID_CLASSES) - 1
iou_subset = np.nanmean(confmat.ious[class_subset])
acc_subset = np.nanmean(confmat.accs[class_subset])
print(f'iou: {iou_subset:.3f}, acc: {acc_subset:.3f}')